# 分数：0.28

# 库

In [43]:
import datetime
import copy
import torch
import dgl
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle
import chardet
import os
import time
import logging
import torch.nn.functional as F
from dgl.dataloading import BlockSampler, NodeDataLoader, MultiLayerFullNeighborSampler
from torch.optim.lr_scheduler import MultiStepLR
from sklearn.model_selection import StratifiedKFold, KFold
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from dgl.nn import SAGEConv,GraphConv

# 换cuda

In [44]:
# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 建图

In [45]:
def csv_reader(data_path):
    with open(data_path, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']
    df_data = pd.read_csv(os.path.join(data_path), encoding=encoding)
    return df_data

In [46]:
def build_graph(ifLoadData, graphPath, dataNodePath, dataEdgePath,isTrain):
    if(ifLoadData):
        with open(graphPath, 'rb') as file:
            graph_list = pickle.load(file)
    else:
        graph_list = []
        node_feats = csv_reader(dataNodePath)
        edge_feats = csv_reader(dataEdgePath)
        
        for c_name, c_df in tqdm(node_feats.groupby('date_id'), desc='Building Graphs'):
            date_src, date_tgt, date_edge_feat = [], [], []
            date_edge = edge_feats[edge_feats['date_id'] == c_name]
            # 在一天的数据里遍历所有的点，建边
            # c_name是分组的日期值，c_df是组内的数据（包含日期）
            for i in tqdm(range(len(c_df)), desc='Date:{}'.format(c_name)):
                now_node = c_df.iloc[i] # 这个日期里的第i个数据，有所有种类的数据
                src = now_node['geohash_id'] # 这行数据是属于哪个点的
                edge_feat = date_edge[date_edge['geohash6_point1'] == src] # 与这个点有关的所有的边
                # 是有向边
                tgt = edge_feat['geohash6_point2'] # 有关边的target点
                date_src.extend([src for _ in range(len(tgt))]) # 产生对应数量的边的src
                date_tgt.extend(tgt) # 加入对应的tgt
                date_edge_feat.extend(pd.concat([edge_feat['F_1'], edge_feat['F_2']])) # 添加边的特征
            date_src = np.array(date_src)
            date_tgt = np.array(date_tgt)
            now_graph = dgl.graph((np.concatenate([np.array(date_src), np.array(date_tgt)], axis=0), np.concatenate([np.array(date_tgt), np.array(date_src)], axis=0)))

            scaler = StandardScaler() 

            if isTrain:
                nodes_data = c_df.drop(['geohash_id', 'date_id', 'active_index', 'consume_index'], axis=1)
            else:
                nodes_data = c_df.drop(['geohash_id', 'date_id'], axis=1)
            
            now_graph.edata['feat'] = torch.from_numpy(scaler.fit_transform(np.concatenate([np.array(date_edge_feat), np.array(date_edge_feat)], axis=0).reshape(-1,2))).float()
            
            now_graph.ndata['feat'] = torch.from_numpy(scaler.fit_transform(nodes_data.values)).float()

            if isTrain:
                now_graph.ndata['active'] = torch.from_numpy(c_df['active_index'].values).float()
                
                now_graph.ndata['consume'] = torch.from_numpy(c_df['consume_index'].values).float()

            graph_list.append(now_graph)

        # # 全部处理完后划分数据集
        # index = list(range(graph_list[0].number_of_nodes()))
        # train_idx, test_idx = train_test_split(index, test_size=args['testSize'], random_state=2, shuffle=True)

        with open(graphPath, 'wb') as file:
            pickle.dump(graph_list, file)

    # return graph_list, train_idx, test_idx
    return graph_list

## train的数据

In [47]:
ifLoadData=True
graphPath="../data/graphs.pkl"
dataNodePath="../data/pre_train_90.csv"
dataEdgePath="../data/pre_edge_90.csv"
isTrain=True

graph_list  = build_graph(ifLoadData, graphPath, dataNodePath, dataEdgePath,isTrain)

In [48]:
graphs = [graph.to(device) for graph in graph_list] # 按天将图图取出来，就是src和tgt
features = [graph.ndata['feat'].to(device) for graph in graph_list] # 每天(图)点的特征
edge_features=[graph.edata['feat'].to(device) for graph in graph_list] # 每天(图)里的边的特征

In [49]:
print(f"len_graph: {len(graphs)},\nlen_feats: {len(features)}")

len_graph: 90,
len_feats: 90


## predict所需要的数据

In [50]:
ifLoadData_test=True
graphPath_test="../data/graphs_test.pkl"
dataNodePath_test="../data/pre_node_test_4_A.csv"
dataEdgePath_test="../data/pre_edge_test_4_A.csv"
isTrain_test=False

graph_list_test  = build_graph(ifLoadData_test, graphPath_test, dataNodePath_test, dataEdgePath_test,isTrain_test)

In [51]:
graphs_test = copy.deepcopy(graphs)
graphs_test.extend([graph.to(device) for graph in graph_list_test]) # 按天将图图取出来，就是src和tgt

features_test = copy.deepcopy(features) 
features_test.extend([graph.ndata['feat'].to(device) for graph in graph_list_test]) # 每天(图)点的特征

edge_features_test = copy.deepcopy(edge_features)
edge_features_test.extend([graph.edata['feat'].to(device) for graph in graph_list_test]) # 每天(图)里的边的特征

# 拉取label

In [52]:
active_index_train, consume_index_train, active_index_test, consume_index_test = [], [], [], []

for graph in graphs:
    active_index_train.append(graph.ndata['active'])
    consume_index_train.append(graph.ndata['consume'])

# [90,1140,2]
print(f"len_label: {len(active_index_train)}")
print(active_index_train)


len_label: 90
[tensor([69.3060, 67.5410, 63.1170,  ..., 65.4980, 73.1520, 72.7780],
       device='cuda:0'), tensor([68.8810, 65.3430, 60.3830,  ..., 64.0170, 72.9140, 70.4900],
       device='cuda:0'), tensor([69.7380, 63.0020, 61.3210,  ..., 64.8800, 72.3910, 75.8860],
       device='cuda:0'), tensor([68.7210, 65.7030, 60.8200,  ..., 66.1420, 73.7120, 72.0960],
       device='cuda:0'), tensor([69.9600, 67.0040, 62.9880,  ..., 66.2720, 72.3940, 72.1170],
       device='cuda:0'), tensor([68.1230, 66.2740, 61.7690,  ..., 65.8640, 72.7230, 70.9290],
       device='cuda:0'), tensor([70.6340, 69.3050, 63.6920,  ..., 67.4300, 72.9020, 72.4490],
       device='cuda:0'), tensor([70.5030, 65.8700, 62.1060,  ..., 66.3270, 72.2050, 73.2050],
       device='cuda:0'), tensor([69.8910, 67.8490, 61.5820,  ..., 67.6790, 73.6380, 72.4870],
       device='cuda:0'), tensor([69.2450, 66.6190, 59.4940,  ..., 66.1200, 72.7610, 72.1920],
       device='cuda:0'), tensor([70.5150, 75.6140, 64.4090,  ..., 64.7

In [53]:
active_index_train = torch.stack(active_index_train)
consume_index_train = torch.stack(consume_index_train)
target_train = torch.cat([active_index_train.unsqueeze(2), consume_index_train.unsqueeze(2)], dim=-1).cpu()

In [54]:
target_train.shape

torch.Size([90, 1140, 2])

# 搭模型

In [55]:
class GraphSAGELayer(nn.Module):
    def __init__(self, in_feats, out_feats, device):
        super(GraphSAGELayer, self).__init__()
        # self.sage = SAGEConv(in_feats, out_feats, 'mean').to(device)
        self.sage = GraphConv(in_feats, out_feats,allow_zero_in_degree=True).to(device)
        self.norm = nn.BatchNorm1d(out_feats).to(device)

    def forward(self, g, features):
        h = self.sage(g, features)
        return (F.relu(h))

In [56]:
class GraphSAGELSTMModel(nn.Module):
    def __init__(self, in_feats, sage_hidden_feats, lstm_hidden_size, mlp_hidden_size, num_layers, device):
        super().__init__()
        self.num_layers=num_layers
        self.lstm_hidden_size = lstm_hidden_size
        self.graphsage = GraphSAGELayer(in_feats, sage_hidden_feats, device)
        self.lstm = nn.LSTM(in_feats, lstm_hidden_size, num_layers).to(device)
        self.fc = nn.Sequential(
            nn.Linear(sage_hidden_feats + lstm_hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 2)
        ).to(device)
        self.norm = nn.BatchNorm1d(self.lstm_hidden_size).to(device)

    def forward(self, graphs, feats):
        # GraphSAGE 处理图数据
        # sage_out维度[num_graph, num_node, features]=[90,1140,hidden_sage]
        sage_out = [self.graphsage(g, feature) for g, feature in zip(graphs, feats)] 
        sage_out = torch.stack(sage_out)
        
        # sage_out = torch.stack(sage_out1, dim=1)  # 维度：[node_num, Time_step, hidden_feats]

        features=torch.stack(feats,dim=0)

        # h0,c0形状[num_graph/time, num_node, hidden_lstm]
        h0=Variable(torch.zeros(self.num_layers,features.shape[1],self.lstm_hidden_size)).to(device)
        c0=Variable(torch.zeros(self.num_layers,features.shape[1],self.lstm_hidden_size)).to(device)
        lstm_out, (hn,cn) = self.lstm(features,(h0,c0))
        # lstm_out_reshaped = lstm_out.reshape(-1, self.lstm_hidden_size)
        
        # [num_graph/time, num_node, hidden_lstm+hidden_sage]

        cat_sage_lstm=torch.cat((sage_out,lstm_out),dim=2) 
        # lstm_out = lstm_out_result.reshape(lstm_out.shape[0],lstm_out.shape[1], lstm_out.shape[2])

        # 维度[num_graph/time, num_node, 2]
        act_cos = self.fc(cat_sage_lstm)

        return act_cos

# 训练参数设置

In [57]:
loss_fn=nn.MSELoss()

model=GraphSAGELSTMModel(
    in_feats=35,
    sage_hidden_feats=32,
    lstm_hidden_size=32,
    mlp_hidden_size=128,
    num_layers=1,
    device=device
)

In [58]:
lr=0.01
epochs=100

optimizer = optim.Adam(model.parameters(), lr=lr)

# 开始训练

In [59]:
tqdm_iter = tqdm(range(epochs), desc='Epochs')
for epoch in tqdm_iter:
    train_loss_list = []
    model.train()
    predict = model(graphs, features).cpu()
    train_loss = torch.sqrt(loss_fn(predict, target_train))  # [Time_step*nodes_num, 2]
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    train_loss_list.append(train_loss.cpu().detach().numpy())

    # 使用 set_postfix 更新进度条后缀
    tqdm_iter.set_postfix({'loss': train_loss.item()}, refresh=True)

Epochs: 100%|██████████| 100/100 [00:20<00:00,  4.81it/s, loss=3.79]


# 开始预测

In [60]:
model.eval()
predict = model(graphs_test, features_test).cpu()
train_loss = torch.sqrt(loss_fn(predict[:90], target_train))  # [Time_step*nodes_num, 2]

print(train_loss)

tensor(3.7592, grad_fn=<SqrtBackward0>)


In [61]:
predict_test=torch.transpose(predict[-4:],0,1)


In [62]:
predict_test.shape

torch.Size([1140, 4, 2])

In [63]:
predict_array=predict_test.detach().numpy().reshape((-1,2))

In [64]:
df_other_info = pd.read_csv("../data/A/node_test_4_A.csv")

id=df_other_info["geohash_id"]
date=df_other_info["date_id"]

In [65]:
df_result = pd.DataFrame({
    "geohash_id": id,
    "consumption_level":predict_array[:, 1],
    "activity_level":predict_array[:, 0],
    "date_id": date
})

# 将合并后的DataFrame保存为新的CSV文件
df_result.to_csv("reslut-3G.csv",sep='\t', index=False)